In [ ]:
# 🚀 CREDIT RISK CLASSIFICATION TRAINING - FIXED VERSION
# ==================================================================================
# ✅ Uses PREPROCESSED data from data_preparation.py output
# ✅ Fully integrated with pipeline_config.yml and experiments.yml
# ✅ Handles PySpark Vector format from preprocessing
# ✅ Logs all experiments to MLflow
# ==================================================================================

%pip install scikit-learn pyyaml

import mlflow
import yaml
import numpy as np
import pandas as pd
import warnings
import time
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, classification_report
)
from mlflow.models.signature import infer_signature
from pyspark.sql import SparkSession
from pyspark.ml.linalg import VectorUDT, Vectors

warnings.filterwarnings("ignore")

print("=" * 80)
print("🚀 CREDIT RISK CLASSIFICATION TRAINING PIPELINE")
print("=" * 80)
 
# ✅ LOAD PIPELINE CONFIGURATION

print("\n📋 Step 1: Loading pipeline configuration...")

try:
    with open("pipeline_config.yml", "r") as f:
        pipeline_cfg = yaml.safe_load(f)

    # Extract configuration values
    MODEL_TYPE = pipeline_cfg["model"]["type"]
    CATALOG = pipeline_cfg["model"]["catalog"]
    SCHEMA = pipeline_cfg["model"]["schema"]
    BASE_NAME = pipeline_cfg["model"]["base_name"]
    
    EXPERIMENT_NAME = pipeline_cfg["experiment"]["name"]
    MODEL_ARTIFACT_PATH = pipeline_cfg["experiment"]["artifact_path"]
    RUN_NAME_PREFIX = pipeline_cfg["experiment"]["run_name_prefix"]

    # ✅ FIXED: Use preprocessed table instead of raw data
    PREPROCESSED_TABLE = pipeline_cfg["data"]["preprocessed_table"]
    LABEL_COL = "label"  # Preprocessed data has 'features' and 'label' columns
    
    # Data split config
    TEST_SIZE = pipeline_cfg["data"]["split"]["test_size"]
    RANDOM_STATE = pipeline_cfg["data"]["split"]["random_state"]
    STRATIFY = pipeline_cfg["data"]["split"]["stratify"]

    # Metrics configuration
    METRICS_CONFIG = pipeline_cfg["metrics"]["classification"]
    PRIMARY_METRIC = METRICS_CONFIG["primary_metric"]
    DIRECTION = METRICS_CONFIG["direction"]
    TRACKED_METRICS = METRICS_CONFIG["tracked_metrics"]
    THRESHOLD_METRICS = METRICS_CONFIG["threshold_metrics"]

    print(f"✅ Pipeline configuration loaded successfully!")
    print(f"\n📊 Configuration Summary:")
    print(f"   Model Type: {MODEL_TYPE.upper()}")
    print(f"   Experiment: {EXPERIMENT_NAME}")
    print(f"   Preprocessed Table: {PREPROCESSED_TABLE}")
    print(f"   Primary Metric: {PRIMARY_METRIC}")
    print(f"   Test Split: {TEST_SIZE * 100}%")

except FileNotFoundError:
    print("❌ ERROR: pipeline_config.yml not found!")
    print("💡 Please ensure pipeline_config.yml is in the notebook directory")
    raise
except Exception as e:
    print(f"❌ ERROR loading pipeline configuration: {e}")
    raise

print("=" * 80)
 
# ✅ LOAD EXPERIMENT CONFIGURATIONS (experiments.yml)

def load_experiment_configs(path="experiments.yml"):
    """Load Random Forest hyperparameter configurations"""
    print(f"\n📄 Step 2: Loading experiment configurations from {path}...")
    
    try:
        with open(path, "r") as f:
            config = yaml.safe_load(f)

        num_experiments = len(config["experiments"])
        print(f"✅ Found {num_experiments} experiment configuration(s):")
        
        for i, exp in enumerate(config["experiments"], 1):
            print(f"   {i}. {exp['name']}")
        
        return config

    except FileNotFoundError:
        print(f"❌ ERROR: {path} not found!")
        print("💡 Please create experiments.yml with Random Forest configurations")
        raise
    except Exception as e:
        print(f"❌ ERROR loading experiment configs: {e}")
        raise
 
# ✅ CONVERT SPARK VECTOR TO NUMPY ARRAY

def vector_to_array(v):
    """Convert PySpark ML Vector to numpy array"""
    if v is None:
        return None
    return v.toArray() if hasattr(v, 'toArray') else np.array(v)
 
# ✅ DATA LOADING FROM PREPROCESSED DELTA TABLE

def load_preprocessed_data(spark):
    """
    Load preprocessed data from Delta table
    Handles PySpark Vector format from data_preparation.py
    
    Returns: X (features), y (labels) as numpy arrays
    """
    print(f"\n📦 Step 3: Loading PREPROCESSED data from Delta table...")
    print(f"   Table: {PREPROCESSED_TABLE}")
    
    try:
        # Read from Delta
        df = spark.read.format("delta").table(PREPROCESSED_TABLE)
        
        # Check schema
        print(f"\n📊 Schema:")
        df.printSchema()
        
        # Convert to pandas
        df_pd = df.toPandas()
        
        print(f"\n✅ Data loaded successfully!")
        print(f"   Total samples: {len(df_pd):,}")
        print(f"   Columns: {list(df_pd.columns)}")
        
        # Extract features and labels
        # 'features' column is a PySpark Vector, need to convert to numpy
        print(f"\n🔄 Converting PySpark Vectors to numpy arrays...")
        
        X = np.array([vector_to_array(row) for row in df_pd['features']])
        y = df_pd['label'].values
        
        print(f"✅ Conversion complete!")
        print(f"   Features shape: {X.shape}")
        print(f"   Labels shape: {y.shape}")
        
        # Check label distribution
        unique_labels, counts = np.unique(y, return_counts=True)
        print(f"\n📊 Label distribution:")
        for label, count in zip(unique_labels, counts):
            label_name = "Default" if label == 1 else "No Default"
            print(f"      Class {int(label)} ({label_name}): {count:,} ({count / len(y) * 100:.1f}%)")
        
        return X, y

    except Exception as e:
        print(f"❌ Failed to load data from '{PREPROCESSED_TABLE}': {e}")
        print("💡 Make sure data_preparation.py has been run successfully")
        print("💡 Verify the table exists: workspace.ml_credit_risk.credit_risk_preprocessed")
        raise
 
# ✅ TRAIN SINGLE EXPERIMENT

def train_single_experiment(X, y, params, run_name):
    """
    Train a single Random Forest Classifier configuration
    
    Args:
        X: Feature matrix (numpy array from preprocessed data)
        y: Target labels
        params: Model hyperparameters from experiments.yml
        run_name: Name for MLflow run
    
    Returns:
        run_id: MLflow run ID
        metrics_dict: Dictionary of evaluation metrics
    """
    print(f"\n{'='*70}")
    print(f"🔁 Training Experiment: {run_name}")
    print(f"{'='*70}")
    print(f"📝 Hyperparameters:")
    for k, v in params.items():
        print(f"   {k}: {v}")

    # Split data (stratified for imbalanced classes)
    stratify_param = y if STRATIFY else None
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, 
        test_size=TEST_SIZE, 
        random_state=RANDOM_STATE,
        stratify=stratify_param
    )

    print(f"\n📊 Data Split:")
    print(f"   Train samples: {len(X_train):,}")
    print(f"   Test samples: {len(X_test):,}")
    print(f"   Train class 1: {(y_train == 1).sum():,} ({(y_train == 1).sum() / len(y_train) * 100:.1f}%)")
    print(f"   Test class 1: {(y_test == 1).sum():,} ({(y_test == 1).sum() / len(y_test) * 100:.1f}%)")

    # Start MLflow run
    with mlflow.start_run(run_name=run_name) as run:
        run_id = run.info.run_id
        print(f"\n🔖 MLflow Run ID: {run_id}")

        # Log configuration metadata
        mlflow.log_param("model_type", MODEL_TYPE)
        mlflow.log_param("experiment_name", run_name)
        mlflow.log_param("timestamp", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        mlflow.log_param("data_source", PREPROCESSED_TABLE)
        mlflow.log_param("preprocessed", True)
        
        # Log all hyperparameters
        for param_name, param_value in params.items():
            mlflow.log_param(param_name, param_value)
        
        # Log data split info
        mlflow.log_param("train_size", len(X_train))
        mlflow.log_param("test_size", len(X_test))
        mlflow.log_param("stratified_split", STRATIFY)
        mlflow.log_param("num_features", X.shape[1])

        # Train model
        print(f"\n🏋️ Training Random Forest Classifier...")
        start_time = time.time()
        
        model = RandomForestClassifier(
            random_state=RANDOM_STATE,
            **params
        )
        
        model.fit(X_train, y_train)
        train_time = time.time() - start_time
        
        print(f"   ✅ Training completed in {train_time:.2f} seconds")
        mlflow.log_metric("train_time", train_time)

        # Make predictions
        print(f"\n📊 Evaluating model...")
        
        # Training predictions
        start_time = time.time()
        y_train_pred = model.predict(X_train)
        y_train_pred_proba = model.predict_proba(X_train)[:, 1]
        
        # Test predictions
        y_test_pred = model.predict(X_test)
        y_test_pred_proba = model.predict_proba(X_test)[:, 1]
        inference_time = (time.time() - start_time) / len(X_test) * 1000  # ms per sample
        
        mlflow.log_metric("inference_time", inference_time)

        # Calculate metrics
        metrics_dict = {
            # Training metrics
            "train_accuracy": accuracy_score(y_train, y_train_pred),
            
            # Test metrics
            "test_accuracy": accuracy_score(y_test, y_test_pred),
            "test_precision": precision_score(y_test, y_test_pred, zero_division=0),
            "test_recall": recall_score(y_test, y_test_pred, zero_division=0),
            "test_f1": f1_score(y_test, y_test_pred, zero_division=0),
            "test_roc_auc": roc_auc_score(y_test, y_test_pred_proba),
            
            # Performance metrics
            "train_time": train_time,
            "inference_time": inference_time
        }

        # Log all metrics to MLflow
        for metric_name, metric_value in metrics_dict.items():
            mlflow.log_metric(metric_name, metric_value)

        # Print metrics
        print(f"\n✅ Evaluation Results:")
        print(f"   {'Metric':<20} {'Train':<12} {'Test':<12}")
        print(f"   {'-'*44}")
        print(f"   {'Accuracy':<20} {metrics_dict['train_accuracy']:<12.4f} {metrics_dict['test_accuracy']:<12.4f}")
        print(f"   {'Precision':<20} {'-':<12} {metrics_dict['test_precision']:<12.4f}")
        print(f"   {'Recall':<20} {'-':<12} {metrics_dict['test_recall']:<12.4f}")
        print(f"   {'F1 Score':<20} {'-':<12} {metrics_dict['test_f1']:<12.4f}")
        print(f"   {'ROC-AUC':<20} {'-':<12} {metrics_dict['test_roc_auc']:<12.4f}")
        print(f"\n   Training Time: {train_time:.2f}s")
        print(f"   Inference Time: {inference_time:.3f}ms/sample")

        # Confusion Matrix
        cm = confusion_matrix(y_test, y_test_pred)
        print(f"\n   Confusion Matrix:")
        print(f"   [[TN={cm[0,0]:<6} FP={cm[0,1]:<6}]")
        print(f"    [FN={cm[1,0]:<6} TP={cm[1,1]:<6}]]")

        # Check threshold metrics
        print(f"\n🎯 Threshold Check:")
        passes_thresholds = True
        for metric_name, threshold_value in THRESHOLD_METRICS.items():
            actual_value = metrics_dict.get(metric_name)
            if actual_value is not None:
                passes = actual_value >= threshold_value
                status = "✅" if passes else "❌"
                print(f"   {status} {metric_name}: {actual_value:.4f} (threshold: {threshold_value})")
                if not passes:
                    passes_thresholds = False
        
        mlflow.log_param("passes_thresholds", passes_thresholds)

        # Create model signature
        signature = infer_signature(X_train, model.predict(X_train))

        # Log model to MLflow
        print(f"\n💾 Logging model to MLflow...")
        mlflow.sklearn.log_model(
            model,
            artifact_path=MODEL_ARTIFACT_PATH,
            signature=signature,
            registered_model_name=None  # Will register later via evaluation script
        )

        # Log feature importance
        if hasattr(model, 'feature_importances_'):
            feature_importance = pd.DataFrame({
                'feature_index': range(len(model.feature_importances_)),
                'importance': model.feature_importances_
            }).sort_values('importance', ascending=False)
            
            print(f"\n🎯 Top 10 Most Important Features:")
            for idx, row in feature_importance.head(10).iterrows():
                print(f"   Feature {row['feature_index']:<5} Importance: {row['importance']:.4f}")
            
            # Log as artifact
            importance_file = "feature_importance.csv"
            feature_importance.to_csv(importance_file, index=False)
            mlflow.log_artifact(importance_file)

        print(f"\n✅ Experiment '{run_name}' completed successfully!")
        print(f"{'='*70}")

        return run_id, metrics_dict
 
# ✅ MAIN EXECUTION

if __name__ == "__main__":
    
    # Initialize MLflow
    print("\n🔧 Initializing MLflow...")
    try:
        mlflow.set_tracking_uri("databricks")
        mlflow.set_registry_uri("databricks-uc")
        mlflow.set_experiment(EXPERIMENT_NAME)
        print(f"✅ MLflow experiment set: {EXPERIMENT_NAME}")
    except Exception as e:
        print(f"❌ Failed to initialize MLflow: {e}")
        raise

    # Initialize Spark
    print("\n🔧 Initializing Spark...")
    try:
        spark = SparkSession.builder.appName("CreditRiskTraining").getOrCreate()
        print("✅ Spark session created")
    except Exception as e:
        print(f"❌ Failed to initialize Spark: {e}")
        raise

    # Load preprocessed data
    X, y = load_preprocessed_data(spark)

    # Load experiment configurations
    config = load_experiment_configs()

    # Start training all experiments
    print("\n" + "=" * 80)
    print("🚀 STARTING TRAINING RUNS")
    print("=" * 80)
    
    run_results = []
    total_experiments = len(config["experiments"])

    for idx, exp in enumerate(config["experiments"], 1):
        exp_name = exp["name"]
        exp_params = exp["params"]
        
        print(f"\n[Experiment {idx}/{total_experiments}]")
        
        try:
            run_id, metrics = train_single_experiment(
                X, y, 
                exp_params, 
                run_name=f"{RUN_NAME_PREFIX}_{exp_name}"
            )
            
            run_results.append({
                'name': exp_name,
                'run_id': run_id,
                'metrics': metrics,
                'params': exp_params
            })
            
        except Exception as e:
            print(f"❌ Failed to train {exp_name}: {e}")
            print(f"   Continuing with next experiment...")
            continue

    # Display final summary
    print("\n" + "=" * 80)
    print("✅✅✅ ALL TRAINING RUNS COMPLETED ✅✅✅")
    print("=" * 80)
    
    if run_results:
        print(f"\n📊 Training Summary ({len(run_results)}/{total_experiments} successful):")
        print(f"\n{'Rank':<6} {'Experiment':<35} {PRIMARY_METRIC.upper():<12} {'ROC-AUC':<10} {'Run ID':<40}")
        print("-" * 103)
        
        # Sort by primary metric
        if DIRECTION == "maximize":
            sorted_results = sorted(run_results, key=lambda x: x['metrics'][PRIMARY_METRIC], reverse=True)
        else:
            sorted_results = sorted(run_results, key=lambda x: x['metrics'][PRIMARY_METRIC])
        
        for rank, result in enumerate(sorted_results, 1):
            marker = "🏆" if rank == 1 else f"{rank}."
            name = result['name']
            primary_score = result['metrics'][PRIMARY_METRIC]
            roc_auc = result['metrics']['test_roc_auc']
            run_id = result['run_id']
            
            print(f"{marker:<6} {name:<35} {primary_score:<12.4f} {roc_auc:<10.4f} {run_id}")
        
        # Highlight best model
        best = sorted_results[0]
        print("\n" + "=" * 80)
        print("🏆 BEST MODEL FROM THIS TRAINING SESSION")
        print("=" * 80)
        print(f"   Name: {best['name']}")
        print(f"   {PRIMARY_METRIC.upper()}: {best['metrics'][PRIMARY_METRIC]:.4f}")
        print(f"   ROC-AUC: {best['metrics']['test_roc_auc']:.4f}")
        print(f"   Accuracy: {best['metrics']['test_accuracy']:.4f}")
        print(f"   Recall: {best['metrics']['test_recall']:.4f}")
        print(f"   Precision: {best['metrics']['test_precision']:.4f}")
        print(f"   Run ID: {best['run_id']}")
        print("\n   Key Hyperparameters:")
        for k in ['n_estimators', 'max_depth', 'min_samples_split', 'class_weight']:
            if k in best['params']:
                print(f"      {k}: {best['params'][k]}")
        print("=" * 80)
        
    else:
        print("\n⚠️ No successful training runs completed")
        print("💡 Check errors above and fix configurations")

    # Next steps
    print("\n📌 Next Steps:")
    print("   1. Run evaluation.py to evaluate and register best models")
    print("   2. Models passing duplicate check will be registered")
    print("   3. Continue with UAT → Production promotion")
    
    print(f"\n💡 Note:")
    print(f"   All {len(run_results)} models logged to: {EXPERIMENT_NAME}")
    print(f"   Evaluation will compare based on: {PRIMARY_METRIC}")
    print(f"   Duplicate detection tolerance: 0.001")
    
    # Save task values for workflow
    try:
        dbutils.jobs.taskValues.set(key="model_type", value=MODEL_TYPE)
        dbutils.jobs.taskValues.set(key="experiment_name", value=EXPERIMENT_NAME)
        dbutils.jobs.taskValues.set(key="num_experiments", value=len(run_results))
        dbutils.jobs.taskValues.set(key="primary_metric", value=PRIMARY_METRIC)
        
        if run_results:
            dbutils.jobs.taskValues.set(
                key="best_score", 
                value=float(sorted_results[0]['metrics'][PRIMARY_METRIC])
            )
            dbutils.jobs.taskValues.set(
                key="best_run_id",
                value=sorted_results[0]['run_id']
            )
        
        print(f"\n✅ Task values saved for workflow automation")
        
    except:
        print(f"\nℹ️ Not running in Databricks workflow - skipping task values")
    
    print("\n🎉 Training pipeline completed successfully!")
    print("=" * 80)